<a href="https://colab.research.google.com/github/imabari/covid19-data/blob/master/mhlw_ocr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# データをダウンロード
!wget https://www.mhlw.go.jp/content/10906000/000668309.pdf -O data.pdf

--2020-09-05 13:01:12--  https://www.mhlw.go.jp/content/10906000/000668309.pdf
Resolving www.mhlw.go.jp (www.mhlw.go.jp)... 172.224.198.160, 2600:1402:2000:183::3194, 2600:1402:2000:18a::3194
Connecting to www.mhlw.go.jp (www.mhlw.go.jp)|172.224.198.160|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 352994 (345K) [application/pdf]
Saving to: ‘data.pdf’

data.pdf            100%[===================>] 344.72K  --.-KB/s    in 0.05s   

2020-09-05 13:01:12 (6.19 MB/s) - ‘data.pdf’ saved [352994/352994]



In [2]:
# PDFBOXをダウンロード
!wget https://www-eu.apache.org/dist/pdfbox/2.0.21/pdfbox-app-2.0.21.jar -O pdfbox-app.jar

--2020-09-05 13:01:12--  https://www-eu.apache.org/dist/pdfbox/2.0.21/pdfbox-app-2.0.21.jar
Resolving www-eu.apache.org (www-eu.apache.org)... 95.216.24.32, 2a01:4f9:2a:185f::2
Connecting to www-eu.apache.org (www-eu.apache.org)|95.216.24.32|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://downloads.apache.org/pdfbox/2.0.21/pdfbox-app-2.0.21.jar [following]
--2020-09-05 13:01:13--  https://downloads.apache.org/pdfbox/2.0.21/pdfbox-app-2.0.21.jar
Resolving downloads.apache.org (downloads.apache.org)... 88.99.95.219, 2a01:4f8:10a:201a::2
Connecting to downloads.apache.org (downloads.apache.org)|88.99.95.219|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9897381 (9.4M) [application/java-archive]
Saving to: ‘pdfbox-app.jar’

pdfbox-app.jar      100%[===================>]   9.44M  8.97MB/s    in 1.1s    

2020-09-05 13:01:14 (8.97 MB/s) - ‘pdfbox-app.jar’ saved [9897381/9897381]



In [3]:
# PDFから画像に変換
!java -jar pdfbox-app.jar PDFToImage -imageType png -dpi 300 data.pdf

In [4]:
# !java -jar pdfbox-app.jar ExtractImages data.pdf

In [5]:
!add-apt-repository ppa:alex-p/tesseract-ocr -y

!apt update

!apt install tesseract-ocr
!apt install libtesseract-dev

!tesseract -v

!apt install tesseract-ocr-jpn  tesseract-ocr-jpn-vert
!apt install tesseract-ocr-script-jpan tesseract-ocr-script-jpan-vert

!tesseract --list-langs
!pip install pytesseract

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/alex-p/tesseract-ocr/ubuntu bionic InRelease [15.4 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:12 https://cloud.r-p

In [6]:
import re

import cv2
import numpy as np
import pytesseract

In [7]:
from google.colab.patches import cv2_imshow

参考：[OpenCVでの表のセルの認識方法](https://teratail.com/questions/151317)

In [8]:
img = cv2.imread("data1.png")

# BGR -> グレースケール
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

gray[gray > 220] = 255
cv2.imwrite("gray.png", gray)

# エッジ抽出 (Canny)
edges = cv2.Canny(gray, 1, 100, apertureSize=3)
cv2.imwrite("edges.png", edges)

# 膨張処理
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
edges = cv2.dilate(edges, kernel)

# 輪郭抽出
contours, hierarchy = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

In [9]:
# 面積でフィルタリング
rects = []

test = []

for cnt, hrchy in zip(contours, hierarchy[0]):
    if cv2.contourArea(cnt) < 2700:
        continue  # 面積が小さいものは除く
    if hrchy[3] == -1:
        continue  # ルートノードは除く

    # 輪郭を囲む長方形を計算する。
    rect = cv2.minAreaRect(cnt)
    rect_points = cv2.boxPoints(rect).astype(int)

    # 座標
    x_max, y_max = np.amax(rect_points.T, axis=1)
    x_min, y_min = np.amin(rect_points.T, axis=1)

    rects.append([y_min, y_max , x_min, x_max])

In [10]:
# x-y 順でソート
rects = sorted(rects, key=lambda x: (x[1], x[2]))

data = []

for i, rect in enumerate(rects):

    y_min, y_max , x_min, x_max = rect

    # 各座標ごとに切り出し
    dst = gray[y_min + 1 : y_max - 1, x_min + 5 : x_max]
    # cv2.imwrite(f"gray{i:03}.png", dst)

    txt = (
        pytesseract.image_to_string(dst, lang="jpn", config="--psm 6")
        .strip()
        .replace(",", "")
        .replace(".", "")
        .replace(" ", "")
    )
    data.append(txt)

    # print("rect:\n", rect)

In [11]:
cv2.imwrite("img.png", img)

True

In [12]:
import pandas as pd

In [13]:
df = pd.DataFrame(rects, columns=["y1", "y2", "x1", "x2"])

In [14]:
df["text"] = data

In [15]:
t1 = set(df["x1"].unique()) & set(df["x1"].unique() + 1)

for i in t1:
    df["x1"] = df["x1"].replace(i, i - 1)

In [16]:
t2 = set(df["y2"].unique()) & set(df["y2"].unique() + 1)

for i in t2:
    df["y2"] = df["y2"].replace(i, i - 1)

In [17]:
pv = df.pivot_table(index="y2", columns="x1", values="text", aggfunc=lambda x: " ".join(str(v) for v in x))

In [18]:
pv.dropna(thresh=7, inplace=True)

In [19]:
pv.dropna(how="all", axis=1, inplace=True)

In [20]:
pv.to_csv("data.csv", index=False, header=False)